# Part 1: Convolutions

In this first part you will see what convolutions do and how they work. 

### Load the Python libraries

Let us start by loading the necessary Python libraries and set a few parameters for the notebook. the `misc` element from `scipy` will allow us to do some elementary image manipulation. 

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# for elementary image loading
import imageio as iio

# specifies the default figure size for this notebook
plt.rcParams['figure.figsize'] = (10, 10)

# specifies the default color map
plt.rcParams['image.cmap'] = 'gray'

### Import an image

To explore how convolutions work, you will use the portrait of [Grace Hopper](https://en.wikipedia.org/wiki/Grace_Hopper).
Use the `imread` function from `imageio` to load the image. ([imread documentation](https://imageio.readthedocs.io/en/stable/userapi.html?highlight=imread#imageio.imread).

The image corresponds to a matrix of dimension H x W x C (height by width by channels). Channels are used for color images. Use mean over the final channels dimensions to convert to a grey-scale image of H x W (height by width), where each entry corresponds to a pixel value between 0 (black) and 255 (white).

Use `imshow` from `plt` to display the image. 
Since it's a matrix, you can use standard numpy style indexing to only show a region.
Show the first 200 lines and the first 600 columns.

In [ ]:
# Load the grace_hopper.jpg image from the data folder
# convert to grayscale


In [ ]:
# Show the image


In [ ]:
# Show another figure with only the first 200 rows / 600 cols


Alternatively, you can view the values of the pixels directly, for example select the first five rows and columns (**top left corner**) and show the corresponding matrix. 

In [ ]:
# Print the pixel values of a region in the top left corner


### Define and apply a convolution function

Now lets define a convolution function. First you must define a function which traverses the image to apply the convolution at every point and returns the result in a filtered image. Calculating the size of the filtered image along each dimension can be a little tricky, the formula is: 

                         (Size of the filtered image) = (input image size) - (filter size) + 1

Let us start by implementing the `convolve` function. It takes as input an image and a filter matrix, and returns
the output of applying the filter at each position in the image through a function `multiply_sum`. 

The function `multiply_sum` corresponds to the following operation for two matrices $A$ and $B$ of identical dimensions:

$$
\text{ multiply\_sum }(A, B) = \sum_{i,j} A_{ij}B_{ij}
$$

i.e. you form a matrix $C$ with entries corresponding to the entry-wise products and you sum across $C$

**Note**: the implentations here are computationally inefficient (and you will see that applying it on the image takes a second). In practice, when dealing with thousands of images, you really don't want sub-optimal operations which is why libraries like Tensorflow hide away *a lot* of optimisations to make such operations as quick as possible and leverage the hardware that is available to you (e.g.: GPU).

In [ ]:
# add your code to define the multiply_sum function (check that it works on a simple example)


The function below defines the convolution operation, go through the code and make sure it makes sense to you what is happening (there is a bit of fiddling required to apply the operations at the right place and store the results appropriately)

In [ ]:
# Convolution function
def convolve(image, filter_matrix):
    
    # get the dimension of the filter
    filter_height = filter_matrix.shape[0]
    filter_width  = filter_matrix.shape[1]
    
    # allocate an empty array for the filtered image using the formula
    # this is the array we'll use to store the result of the convolution
    filtered_image = np.ndarray(shape=(image.shape[0] - filter_height + 1, 
                                       image.shape[1] - filter_width + 1))
    
    # go through rows
    for row in range(filtered_image.shape[0]):
        # go through columns
        for col in range(filtered_image.shape[1]):
            # select a local patch of the image
            patch = image[row:(row + filter_height), 
                          col:(col + filter_width)]

            # apply the multiply_sum operation
            ms = multiply_sum(patch, filter_matrix)
            
            # store it at the right location
            filtered_image[row, col] = ms
            
    return filtered_image

An there you have it, a convolution operator! You can apply a filter onto an image and see the result. 

Define a filter matrix corresponding to

$$
\left(\begin{array}{ccc}
    -1&-1&-1\\ 
    2&2&2\\ 
    -1&-1&-1
\end{array}\right)
$$

apply it to GH's portrait and display the result with `plot_filter`: a function we provide below

In [ ]:
def plot_filter(filtered_image, cmap=None):
    if cmap is None:
        cmap = plt.get_cmap('bwr')  # a diverging cmap
    # set vmin = vmax such that 0 is in the centre of the cmap
    vmax = int(
        max(
            np.abs(np.max(filtered_image)),
            np.abs(np.min(filtered_image))
        )
    )
    vmin = -vmax
    im = plt.imshow(filtered_image, vmax=vmax, vmin=vmin, cmap=cmap)
    plt.colorbar(im, fraction=0.1)
    plt.axis('off')

In [ ]:
# First define the 3x3 filter 

# Then apply it to the image using the convolve function defined above


In [ ]:
# Finally show the result using our plot_filter function


### Quiz: What did our filter do?

1) By looking at the image, can you tell what kind of pattern the filter detected?

2) How would you design a filter which detects vertical edges?

3) What would the following filter do: ([Prewitt operator](https://en.wikipedia.org/wiki/Prewitt_operator))

$$
\left(\begin{array}{ccc}
    1&1&1\\ 
    0&0&0\\ 
    -1&-1&-1
\end{array}\right)
$$


how about its transpose? how about if you swap the first and last row?

Try variations until you get an intuition for what these operators do.

In [ ]:
# Define the filter matrix


In [ ]:
# Show the result of convolving the image with this new filter


In [ ]:
# Repeat with the transpose of the filter


In [ ]:
# Repeat with the filter but with its first and last rows swapped


### Convolutions with colour

Very good! But what if we had a coloured image, how would we use that extra information to detect useful patterns? 
The idea is simple: on top of having a set weight for each pixel, we have a set weight for each colour channel within that pixel.
Filters become stacks of kernels (usually 3 for the three channels: R, G, B). 

An example is the following kernel which detects region of the image that are mostly brown.

In [ ]:
brown_filter = np.array(
      [[[ 0.13871045,  0.17157242,  0.12934428], # Red channel
        [ 0.16168842,  0.20229845,  0.14835016],
        [ 0.135694  ,  0.16206263,  0.11727387]],

       [[ 0.04231958,  0.05471011,  0.03167877], # Green channel
        [ 0.0462575 ,  0.06581022,  0.03104937],
        [ 0.04185439,  0.04734124,  0.02087744]],

       [[-0.15704881, -0.16666673, -0.16600266], # Blue channel
        [-0.17439997, -0.17757156, -0.18760149],
        [-0.15435153, -0.17037505, -0.17269668]]])

print(brown_filter.shape)

The **first** dimension corresponds the three channels (R, G, B) (try `brown_filter[1, :, :]` for the filter values corresponding to the green channel). 

Looking at the values, you can see that the filter responds to regions that are red (positive values, reasonably large), a little bit to the green values (positive values, quite small), and not at all to regions that are blue (negative values). 

To see it in practice, use `imread` from `imageio` without `as_gray=True` to load the coloured image of Grace Hopper.

Show the image and its dimensions

In [ ]:
# Load and display the coloured image of Grace Hopper

# Show the shape of the image


Now we would like to apply the Brown filter and see the result. 
One thing needs to be done, it's a bit annoying but it happens *all the time* in CNNs (and other non-trivial NNs): you need to adjust dimensions. 
Currently, it is the **first** dimension of the brown filter that corresponds to the colour channels while you saw that it is the **last** dimension of GH's image that correspond to the channels. 

Therefore you need to re-arrange dimensions from 

```
(0, 1, 2) -----> (1, 2, 0)
```

this can be done via the `transpose` method: `array.transpose((1, 2, 0))`.

Adjust the brown filter and apply it. 

In [ ]:
# Adjust the dimensions of the brown filter and apply it to the image of GH


### Quiz

Can you design a filter which will detect the edge from the background (blue) to Grace Hopper’s left shoulder (black).

**Note**: it's good practice to have the weights in your filter sum to 0 and don't forget to re-arrange the dimensions.

In [ ]:
# devise a left_shouler_filter and apply it
